### <b>1. ProGAN<b>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# PixelNorm Layer
# 채널별로 정규화 -> 학습 안정화
class PixelNorm(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x, eps=1e-8):
        # 각 pixel vector의 L2-norm으로 정규화
        return x / torch.sqrt(torch.mean(x ** 2, dim=1, keepdim=True) + eps)

# Equalized Learning Rate 적용 Conv2d
# 학습 안정화를 위한 스케일링 적용
class WSConv2d(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, stride=1, padding=0):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, kernel_size, stride, padding)
        # 논문: He 초기화 후 weight를 정규분포로 초기화
        nn.init.normal_(self.conv.weight)
        # 스케일링 계수 (He initialization과 동일한 분산 사용)
        self.scale = (2 / (in_ch * kernel_size ** 2)) ** 0.5

    def forward(self, x):
        return self.conv(x * self.scale)

# Equalized Learning Rate 적용 Linear
# FC layer version
class WSLinear(nn.Module):
    def __init__(self, in_f, out_f):
        super().__init__()
        self.linear = nn.Linear(in_f, out_f)
        nn.init.normal_(self.linear.weight)
        # He 초기화 기반 스케일링
        self.scale = (2 / in_f) ** 0.5 

    def forward(self, x):
        return self.linear(x * self.scale)


In [2]:
# Generator
# Block 순서 
# Conv → LeakyReLU → PixelNorm → Conv → LeakyReLU → PixelNorm
class GenBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.block = nn.Sequential(
            WSConv2d(in_ch, out_ch, 3, 1, 1), # Equalized LR 적용
            nn.LeakyReLU(0.2),
            PixelNorm(), # Pixel 단위 정규화
            WSConv2d(out_ch, out_ch, 3, 1, 1),
            nn.LeakyReLU(0.2),
            PixelNorm(),
        )

    def forward(self, x):
        return self.block(x)

class Generator(nn.Module):
    def __init__(self, z_dim, base_channels=512):
        super().__init__()
        # 초기 fully-connected + reshape 단계:
        # z → 4x4 feature map
        self.initial = nn.Sequential(
            PixelNorm(), # latent 벡터 정규화
            WSLinear(z_dim, base_channels * 4 * 4), # z → 4x4 feature map으로 변환
            nn.LeakyReLU(0.2),
        )
        self.initial_conv = GenBlock(base_channels, base_channels) # 4x4 block
        self.to_rgb_layers = nn.ModuleList([WSConv2d(base_channels, 3, 1, 1, 0)]) # toRGB conv for 4x4
        self.blocks = nn.ModuleList()  # 해상도 증가를 위한 Blocks
        self.channel_map = [base_channels, 256, 128, 64, 32, 16]  # 해상도별 채널 설정

        for i in range(len(self.channel_map) - 1):
            in_ch, out_ch = self.channel_map[i], self.channel_map[i + 1]
            self.blocks.append(GenBlock(in_ch, out_ch)) # 새로운 Block 추가
            self.to_rgb_layers.append(WSConv2d(out_ch, 3, 1, 1, 0)) # toRGB conv

    def fade_in(self, alpha, old, new):
        # fade-in: 기존 출력을 조금씩 섞으면서 새로운 해상도에 적응
        return torch.tanh(alpha * new + (1 - alpha) * old)

    def forward(self, z, step, alpha):
        out = self.initial(z).view(z.shape[0], self.channel_map[0], 4, 4) # z를 4x4 feature로 변환
        out = self.initial_conv(out)

        if step == 0:
            return torch.tanh(self.to_rgb_layers[0](out)) # 초기 단계는 바로 toRGB

        for i in range(step): # step 수만큼 업샘플링과 블록 통과
            out = F.interpolate(out, scale_factor=2, mode='bilinear', align_corners=False)
            out = self.blocks[i](out)

        out_new = self.to_rgb_layers[step](out)  # 새 RGB 출력
        out_old = self.to_rgb_layers[step - 1](  # 이전 해상도의 RGB 출력
            F.interpolate(out, scale_factor=0.5) # 절반으로 축소 후 적용
        )
        out_old = F.interpolate(out_old, scale_factor=2, mode='bilinear', align_corners=False)

        return self.fade_in(alpha, out_old, out_new) # 두 출력을 섞어서 return


In [3]:
# Discriminator
# Blocks: Conv → LeakyReLU → Conv → LeakyReLU
class DiscBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.block = nn.Sequential(
            WSConv2d(in_ch, in_ch, 3, 1, 1),
            nn.LeakyReLU(0.2),
            WSConv2d(in_ch, out_ch, 3, 1, 1),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.block(x)

class Discriminator(nn.Module):
    def __init__(self, base_channels=512):
        super().__init__()
        self.channel_map = [base_channels, 256, 128, 64, 32, 16] # 채널 매핑
        self.blocks = nn.ModuleList() # 해상도 단계별 블록
        self.from_rgb = nn.ModuleList([WSConv2d(3, base_channels, 1, 1, 0)])  # 4x4 입력용 fromRGB

        for i in range(len(self.channel_map) - 1):
            in_ch, out_ch = self.channel_map[i + 1], self.channel_map[i]
            self.blocks.append(DiscBlock(in_ch, out_ch)) # 블록 추가
            self.from_rgb.append(WSConv2d(3, in_ch, 1, 1, 0)) # fromRGB 추가

        self.final_block = nn.Sequential(
            nn.AvgPool2d(4), # Global average pooling
            WSConv2d(base_channels + 1, base_channels, 3, 1, 1), # Minibatch stddev 포함 처리
            nn.LeakyReLU(0.2),
            WSConv2d(base_channels, base_channels, 4, 1, 0), # 4x4 → 1x1
            nn.LeakyReLU(0.2),
            WSLinear(base_channels, 1) # 최종 score
        ) 

    def fade_in(self, alpha, old, new):
        return alpha * new + (1 - alpha) * old # fade-in 적용

    def forward(self, x, step, alpha):
        if step == 0:
            x = self.from_rgb[0](x) # 4x4용 fromRGB
            return self.final_block(x)

        downscaled = F.avg_pool2d(x, 2) # 이전 단계 입력 생성
        out_old = self.from_rgb[step - 1](downscaled) # 이전 단계로 변환
        out_new = self.from_rgb[step](x) # 현재 단계로 변환
        out_new = self.blocks[step - 1](out_new) # 블록 통과

        x = self.fade_in(alpha, out_old, out_new)  # 두 출력을 fade-in

        for i in reversed(range(step - 1)):
            x = self.blocks[i](x) # 나머지 블록 통과
            x = F.avg_pool2d(x, 2) # 다운샘플링

        # Minibatch StdDev Trick
        batch_std = torch.std(x, dim=0, keepdim=True).mean() # 배치 단위 표준편차
        batch_std = batch_std.expand(x.size(0), 1, 4, 4) # 차원 맞추기
        x = torch.cat([x, batch_std], dim=1) # 채널에 붙이기

        return self.final_block(x) # 최종 판별 결과


### <b>2. StyleGAN<b>

In [4]:
# Mapping Network : Latent vector z -> ntermediate latent space w로 변환(8-layer MLP)
# Adaptive Instance Normalization (AdaIN): 스타일 벡터 w를 각 레이어에 주입하여 스타일 조절
# Constant Input: z를 직접 convolution 하지 않고, 고정된 learnable constant에서 시작
# Noise Injection: 각 레이어에 독립적인 노이즈를 더해 세부 묘사 제어
# Progressive Growing: ProGAN처럼 해상도를 점진적으로 키움 (optional)

In [5]:
# StyleGAN 기본 구조
import torch
import torch.nn as nn
import torch.nn.functional as F

# 1. PixelNorm
class PixelNorm(nn.Module):
    def forward(self, x, eps=1e-8):
        # 각 픽셀 벡터의 L2 norm으로 정규화 -> 학습 안정화
        return x / torch.sqrt(torch.mean(x ** 2, dim=1, keepdim=True) + eps)

# 2. Mapping Network (z → w)
class MappingNetwork(nn.Module):
    def __init__(self, z_dim, w_dim, num_layers=8):
        super().__init__()
        layers = [PixelNorm()] # z 입력 정규화
        for _ in range(num_layers): # 8개의 fully connected 레이어
            layers.append(nn.Linear(z_dim, w_dim))
            layers.append(nn.LeakyReLU(0.2))
            z_dim = w_dim
        self.mapping = nn.Sequential(*layers) # 전체 mapping 네트워크

    def forward(self, z):
        return self.mapping(z) # z → w 로 변환

# 3. Noise Injection
class NoiseInjection(nn.Module):
    def __init__(self, channels):
        super().__init__()
        # 노이즈를 각 채널에 적용할 때 곱해줄 learnable weight
        self.weight = nn.Parameter(torch.zeros(1, channels, 1, 1))

    def forward(self, x, noise=None):
        if noise is None:
            noise = torch.randn_like(x) # 없으면 랜덤 노이즈 생성
        return x + self.weight * noise # 노이즈 주입

# 4. AdaIN (w 벡터로 style 조절)
class AdaIN(nn.Module):
    def __init__(self, channels, w_dim):
        super().__init__()
        self.scale = nn.Linear(w_dim, channels) # w → scale
        self.bias = nn.Linear(w_dim, channels) # w → bias

    def forward(self, x, w):
        mean = x.mean(dim=(2, 3), keepdim=True) # feature map 평균
        std = x.std(dim=(2, 3), keepdim=True) # feature map 표준편차
        normalized = (x - mean) / (std + 1e-8) # 정규화
        style_scale = self.scale(w).unsqueeze(2).unsqueeze(3) # w로부터 scale
        style_bias = self.bias(w).unsqueeze(2).unsqueeze(3) # w로부터 bias
        return style_scale * normalized + style_bias # 스타일 변환 결과

# 5. StyledConv (Noise + AdaIN + Conv)
class StyledConv(nn.Module):
    def __init__(self, in_ch, out_ch, w_dim, kernel_size=3, padding=1):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, kernel_size, padding=padding)
        self.noise = NoiseInjection(out_ch) # 노이즈 주입 모듈
        self.activation = nn.LeakyReLU(0.2)
        self.adain = AdaIN(out_ch, w_dim) # AdaIN 모듈

    def forward(self, x, w, noise=None):
        x = self.conv(x)
        x = self.noise(x, noise) # 노이즈 추가
        x = self.activation(x)
        x = self.adain(x, w) # AdaIN을 통해 스타일 적용
        return x

In [6]:
# Generator
class Generator(nn.Module):
    def __init__(self, z_dim=512, w_dim=512, channels=[512, 256, 128, 64, 32]):
        super().__init__() 
        self.mapping = MappingNetwork(z_dim, w_dim) # Mapping 네트워크
        self.constant_input = nn.Parameter(torch.randn(1, channels[0], 4, 4)) # learnable constant 시작점

        self.blocks = nn.ModuleList()
        self.to_rgb = nn.ModuleList() # 각 해상도에서 RGB로 변환하는 레이어
 
        in_ch = channels[0]
        for out_ch in channels:
            # StyledConv 2개가 쌍으로 있는 block 추가
            self.blocks.append(nn.ModuleList([
                StyledConv(in_ch, out_ch, w_dim),
                StyledConv(out_ch, out_ch, w_dim)
            ]))
            self.to_rgb.append(nn.Conv2d(out_ch, 3, 1)) # toRGB conv
            in_ch = out_ch

    def forward(self, z, step=0, alpha=1.0, noise=None):
        w = self.mapping(z)  # z → w
        x = self.constant_input.expand(z.shape[0], -1, 4, 4) # constant 입력 복사

        for i in range(step + 1): # 현재 해상도까지 반복
            block = self.blocks[i]
            x = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False) if i > 0 else x
            x = block[0](x, w) # styled conv 1
            x = block[1](x, w) # styled conv 2

        img = self.to_rgb[step](x) # RGB 변환
        return torch.tanh(img) # 출력 [-1, 1]로 정규화


In [7]:
# Discriminator (mirror of generator)

# Minibatch Stddev layer
class MinibatchStdDev(nn.Module):
    def forward(self, x):
        std = x.std(0, keepdim=True).mean() # 배치의 표준편차 평균
        shape = list(x.shape)
        shape[1] = 1
        std_map = std.expand(shape) # 모든 위치에 복사
        return torch.cat([x, std_map], 1)

class DiscBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, in_ch, 3, padding=1)
        self.conv2 = nn.Conv2d(in_ch, out_ch, 3, padding=1)
        self.activation = nn.LeakyReLU(0.2)

    def forward(self, x):
        x = self.activation(self.conv1(x))
        x = self.activation(self.conv2(x))
        return F.avg_pool2d(x, 2) # 다운샘플링

class Discriminator(nn.Module):
    def __init__(self, channels=[512, 256, 128, 64, 32]):
        super().__init__()
        self.from_rgb = nn.ModuleList() # 이미지 → feature map
        self.blocks = nn.ModuleList() 
        in_ch = channels[-1]

        for out_ch in reversed(channels[:-1]):
            self.from_rgb.append(nn.Conv2d(3, in_ch, 1)) # RGB 변환
            self.blocks.append(DiscBlock(in_ch, out_ch))
            in_ch = out_ch

        self.stddev = MinibatchStdDev() # Minibatch stddev
        self.final_conv = nn.Conv2d(channels[0]+1, channels[0], 3, padding=1)
        self.final_fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(channels[0]*4*4, 1) # score 출력
        )

    def forward(self, img, step=0, alpha=1.0):
        x = self.from_rgb[step](img) # RGB → feature
        for i in range(step, -1, -1): # 거꾸로 처리
            x = self.blocks[i](x)
        x = self.stddev(x) # Minibatch stddev 추가
        x = self.final_conv(x)
        return self.final_fc(x) # score 출력


In [8]:
# WGAN-GP Loss
# gradient penalty
def gradient_penalty(d, real, fake, step, alpha):
    batch_size = real.size(0)
    epsilon = torch.rand(batch_size, 1, 1, 1, device=real.device) # 랜덤 보간 계수
    interpolated = real * epsilon + fake * (1 - epsilon) # 중간값
    interpolated.requires_grad_() 

    d_interpolated = d(interpolated, step, alpha) # 판별
    grad = torch.autograd.grad(
        outputs=d_interpolated,
        inputs=interpolated,
        grad_outputs=torch.ones_like(d_interpolated),
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]

    grad = grad.view(grad.size(0), -1) # 2D
    gp = ((grad.norm(2, dim=1) - 1) ** 2).mean()  # L2-norm 기반 penalty
    return gp


### <b>3. StyleGAN2<b>

In [9]:
# 개선사항
# Weight Demodulation: AdaIN 대신 weight demodulation을 통해 스타일 적용
# Blurred Upsample / Downsample: aliasing 제거를 위해 blur kernel 사용
# ResNet-style Skip Connection: Discriminator에 skip 연결 도입
# Progressive Growing 제거: 고정된 해상도에서 바로 학습 가능 (fused training)

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# PixelNorm
class PixelNorm(nn.Module):
    def forward(self, x, eps=1e-8):
        # 채널 단위 평균 제곱값으로 나눠 정규화 -> 학습 안정화
        return x / torch.sqrt(torch.mean(x ** 2, dim=1, keepdim=True) + eps)

# Mapping Network: z → w
class MappingNetwork(nn.Module):
    def __init__(self, z_dim=512, w_dim=512, num_layers=8):
        super().__init__()
        layers = [PixelNorm()] # PixelNorm을 통해 z 정규화
        for _ in range(num_layers):
            layers.append(nn.Linear(z_dim, w_dim)) # FC layer로 latent 변환
            layers.append(nn.LeakyReLU(0.2))
        self.mapping = nn.Sequential(*layers) # Sequential로 묶기

    def forward(self, z):
        return self.mapping(z) # 입력 z → 스타일 벡터 w

# Noise Injection (per-channel learnable scale)
class NoiseInjection(nn.Module):
    def __init__(self, channels):
        super().__init__()
        # 각 채널마다 노이즈의 중요도를 조절하는 learnable weight
        self.weight = nn.Parameter(torch.zeros(1, channels, 1, 1))

    def forward(self, x, noise=None):
        if noise is None:
            noise = torch.randn_like(x) # 랜덤 노이즈 생성
        return x + self.weight * noise # 노이즈 주입

# ModulatedConv2d: 핵심 모듈 (StyleGAN2 핵심)
class ModulatedConv2d(nn.Module):
    def __init__(self, in_ch, out_ch, style_dim, kernel_size=3, demodulate=True):
        super().__init__()
        self.eps = 1e-8
        self.in_ch = in_ch
        self.out_ch = out_ch
        self.kernel_size = kernel_size
        self.demodulate = demodulate

        # Conv weight: (1, out_ch, in_ch, k, k) → 스타일마다 스케일링
        self.weight = nn.Parameter(
            torch.randn(1, out_ch, in_ch, kernel_size, kernel_size)
        )
        # 스타일 벡터를 입력 채널 차원에 맞게 조정하는 FC layer
        self.style = nn.Linear(style_dim, in_ch)

    def forward(self, x, w):
        batch, _, height, width = x.shape
        
        # 스타일 벡터를 input 채널에 맞게 변환 (scale 역할)
        style = self.style(w).view(batch, 1, self.in_ch, 1, 1)
        
        # weight modulation: 스타일로 스케일링
        weight = self.weight * style

        # weight demodulation: 각 out 채널의 분산을 1로 정규화
        if self.demodulate:
            demod = torch.rsqrt((weight ** 2).sum([2, 3, 4]) + self.eps)
            weight = weight * demod.view(batch, self.out_ch, 1, 1, 1)

        # convolution을 위해 weight와 x를 reshape
        weight = weight.view(batch * self.out_ch, self.in_ch, self.kernel_size, self.kernel_size)
        x = x.view(1, batch * self.in_ch, height, width)
        
        # Grouped convolution을 통해 스타일별 독립 처리
        out = F.conv2d(x, weight, padding=self.kernel_size // 2, groups=batch)
        return out.view(batch, self.out_ch, height, width) # 원래 배치 형태로 복원

# StyledConvBlock: ModConv + Noise + Activation
class StyledConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, style_dim):
        super().__init__()
        # 스타일 기반 modulated conv
        self.modconv = ModulatedConv2d(in_ch, out_ch, style_dim)
        self.noise = NoiseInjection(out_ch) # 채널별 노이즈 주입
        self.activation = nn.LeakyReLU(0.2)

    def forward(self, x, w, noise=None):
        x = self.modconv(x, w) # 스타일에 따라 weight modulation된 conv
        x = self.noise(x, noise) # 노이즈 추가
        return self.activation(x)

# Generator
class Generator(nn.Module):
    def __init__(self, z_dim=512, w_dim=512, channels=[512, 256, 128, 64, 32]):
        super().__init__()
        self.mapping = MappingNetwork(z_dim, w_dim) # Mapping 네트워크
        # 처음에 사용할 learnable constant (4x4 feature map)
        self.constant = nn.Parameter(torch.randn(1, channels[0], 4, 4))

        self.blocks = nn.ModuleList() # 스타일 블록 리스트
        self.to_rgb = nn.ModuleList() # RGB로 변환하는 conv 리스트

        in_ch = channels[0]
        for out_ch in channels:
            # 각 해상도별 conv2개 블록 생성
            self.blocks.append(nn.ModuleList([
                StyledConvBlock(in_ch, out_ch, w_dim),
                StyledConvBlock(out_ch, out_ch, w_dim)
            ]))
            self.to_rgb.append(nn.Conv2d(out_ch, 3, kernel_size=1)) # toRGB layer
            in_ch = out_ch

    def forward(self, z, noise=None):
        w = self.mapping(z) # z → w 스타일 벡터 생성
        x = self.constant.expand(z.shape[0], -1, 4, 4) # 배치 크기에 맞게 constant input 복제

        # 해상도 단계별로 업샘플 → 블록 통과
        for i, block in enumerate(self.blocks):
            x = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False) if i > 0 else x
            x = block[0](x, w, noise)
            x = block[1](x, w, noise)
            
        # 최종 RGB 이미지 출력 (3채널)
        img = self.to_rgb[-1](x)
        return torch.tanh(img) # [-1, 1] 범위로 정규화


In [11]:
class DiscResBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, in_ch, 3, padding=1)
        self.conv2 = nn.Conv2d(in_ch, out_ch, 3, padding=1)
        self.skip = nn.Conv2d(in_ch, out_ch, 1)
        self.activation = nn.LeakyReLU(0.2)

    def forward(self, x):
        # Skip connection (Residual 경로)
        # x를 1x1 Conv로 변환하고, 다운샘플링 (Average Pooling)
        residual = F.avg_pool2d(self.skip(x), 2) 
        x = self.activation(self.conv1(x)) # 메인 경로 - 첫 번째 Conv 후 LeakyReLU
        x = self.activation(self.conv2(x))
        x = F.avg_pool2d(x, 2) # 해상도를 줄이기 위해 평균 풀링 적용 (2x down)
        # Residual 연결 + 정규화 (sqrt(2)로 나누는 건 ResNet의 variance-preserving trick)
        return (x + residual) / math.sqrt(2)

class Discriminator(nn.Module):
    def __init__(self, channels=[512, 256, 128, 64, 32]):
        super().__init__()
        # 입력 이미지를 feature map으로 변환하는 1x1 Conv
        self.from_rgb = nn.Conv2d(3, channels[-1], 1) # in_ch → out_ch 블록 추가
        self.blocks = nn.ModuleList() # ResBlock 리스트

        in_ch = channels[-1]
        # 채널을 거꾸로 줄여가며 ResBlock 구성
        for out_ch in reversed(channels[:-1]):
            self.blocks.append(DiscResBlock(in_ch, out_ch))
            in_ch = out_ch # 다음 블록 입력 채널로 설정

        # 마지막 단계: Conv → Flatten → Linear로 score 예측
        self.final = nn.Sequential(
            nn.Conv2d(in_ch, in_ch, 3, padding=1),
            nn.LeakyReLU(0.2),
            nn.Flatten(),
            nn.Linear(in_ch * 4 * 4, 1) # 4x4 feature map → scalar 판별값
        )

    def forward(self, x):
        x = self.from_rgb(x) # RGB 이미지 → feature map (3채널 → channels[-1]
        for block in self.blocks:  # 각 블록을 순서대로 통과
            x = block(x)
        return self.final(x) # 최종 classifier로 score 출력


### <b>4. SAGAN<b>

In [12]:
# Self-Attention Layer: CNN의 지역적인 receptive field 한계를 보완하기 위해 전역적인 정보 상호작용 추가
# Spectral Normalization: Discriminator에 적용 → 학습 안정성 향상
# Residual Block: 안정적인 학습을 위한 ResNet 구조 활용
# BatchNorm 대신 Conditional BatchNorm: Class-aware 이미지 생성에 적합 (optional)

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Self-Attention Layer
class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        # Query, Key: in_channels → in_channels / 8 (차원 축소)
        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, 1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, 1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, 1)  # Value: 원래 채널 크기 유지
         # 학습 가능한 스칼라: attention 출력 비율 조절
        self.gamma = nn.Parameter(torch.zeros(1))  # 학습 가능한 scale 계수

    def forward(self, x):
        B, C, H, W = x.shape # 배치, 채널, 세로, 가로

        # Query, Key, Value 계산
        proj_query = self.query_conv(x).view(B, -1, H * W).permute(0, 2, 1)  # [B, HW, C//8]
        proj_key = self.key_conv(x).view(B, -1, H * W)                       # [B, C//8, HW]
        # Query × Keyᵀ = attention 에너지 (유사도)
        energy = torch.bmm(proj_query, proj_key)                            # [B, HW, HW]
        # softmax로 attention map 생성 (행 기준 정규화)
        attention = F.softmax(energy, dim=-1)  # attention map

        proj_value = self.value_conv(x).view(B, -1, H * W)                  # [B, C, HW]
        # attention을 value에 곱한다.
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))             # [B, C, HW]
        # 다시 spatial map 형태로 reshape
        out = out.view(B, C, H, W)

        # 입력과 attention 결과를 결합 (잔차 연결 + 학습 가능한 스케일링)
        return self.gamma * out + x


In [14]:
# Generator(ex)
# Transposed Conv + Attention
class Generator(nn.Module):
    def __init__(self, z_dim=128, channels=64):
        super().__init__()
        self.net = nn.Sequential(
            # z: (B, z_dim) → (B, ch*8, 4, 4)
            nn.ConvTranspose2d(z_dim, channels * 8, 4, 1, 0),   # 1x1 → 4x4
            nn.BatchNorm2d(channels * 8),
            nn.ReLU(),

            # 4x4 → 8x8
            nn.ConvTranspose2d(channels * 8, channels * 4, 4, 2, 1),  # 4x4 → 8x8
            nn.BatchNorm2d(channels * 4),
            nn.ReLU(),

            # 8x8 → 16x16
            nn.ConvTranspose2d(channels * 4, channels * 2, 4, 2, 1),  # 8x8 → 16x16
            nn.BatchNorm2d(channels * 2),
            nn.ReLU(),

            # 16x16 → 32x32
            nn.ConvTranspose2d(channels * 2, channels, 4, 2, 1),  # 16x16 → 32x32
            nn.BatchNorm2d(channels),
            nn.ReLU(),

            # Self-Attention 삽입: 32x32 해상도에서 전역 상호작용
            SelfAttention(channels),  # Attention 위치: 32x32 해상도
            nn.ConvTranspose2d(channels, 3, 4, 2, 1),  # 32x32 → 64x64
            nn.Tanh()
        )

    def forward(self, z):
        # z: (B, z_dim) → (B, z_dim, 1, 1)로 reshape
        return self.net(z.view(z.size(0), z.size(1), 1, 1))


In [15]:
# Spectral Normalized Conv2d
def SNConv2d(*args, **kwargs):
    return nn.utils.spectral_norm(nn.Conv2d(*args, **kwargs))

In [16]:
# Discriminator (SpectralNorm + Self-Attention) 
class Discriminator(nn.Module):
    def __init__(self, channels=64):
        super().__init__()
        self.net = nn.Sequential(
            # 입력 이미지 → feature map
            SNConv2d(3, channels, 4, 2, 1),       # 64x64 → 32x32
            nn.LeakyReLU(0.2),

            SNConv2d(channels, channels * 2, 4, 2, 1),  # 32x32 → 16x16
            nn.LeakyReLU(0.2),

            SNConv2d(channels * 2, channels * 4, 4, 2, 1),  # 16x16 → 8x8
            nn.LeakyReLU(0.2),

            # # Self-Attention 삽입
            # 8x8 해상도에서 전역 정보 파악
            SelfAttention(channels * 4),  # Attention 위치: 8x8
            SNConv2d(channels * 4, channels * 8, 4, 2, 1),  # 8x8 → 4x4
            nn.LeakyReLU(0.2),

            SNConv2d(channels * 8, 1, 4)  # 4x4 → 1x1 → 스칼라
        )

    def forward(self, x):
        return self.net(x).view(-1)  # 결과 shape: (B, 1, 1, 1) → (B,)로 펼친다.


### <b>5. BigGAN<b>

In [17]:
# Class-conditional GAN:입력에 클래스 정보를 넣어 class-aware 이미지 생성
# self-Attention: SAGAN에서 도입한 전역 정보 교환 방식 채택
# Spectral Normalization: 안정적 학습을 위해 모든 Conv/Linear에 적용
# Shared Embedding: 클래스 임베딩을 여러 레이어에서 공유 (G에서 style modulation처럼 사용)
# Orthogonal Regularization: Discriminator의 안정화를 위

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConditionalBatchNorm(nn.Module):
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.bn = nn.BatchNorm2d(num_features, affine=False) # 일반 BatchNorm, gamma/beta는 제거
        # 클래스에 따라 gamma와 beta를 학습하는 embedding
        self.gamma_embed = nn.Embedding(num_classes, num_features)
        self.beta_embed = nn.Embedding(num_classes, num_features)

        # 임베딩 초기화
        # 초기화: gamma=1, beta=0 (초기에는 일반 BN처럼 동작)
        nn.init.ones_(self.gamma_embed.weight)
        nn.init.zeros_(self.beta_embed.weight)

    def forward(self, x, y):
        out = self.bn(x) # 일반 BatchNorm 수행
        gamma = self.gamma_embed(y).unsqueeze(2).unsqueeze(3)  # [B, C] → [B, C, 1, 1]
        beta = self.beta_embed(y).unsqueeze(2).unsqueeze(3)
        return gamma * out + beta # 클래스 조건에 따라 gamma/beta 적용


In [19]:
class SelfAttention(nn.Module): # SAGAN 구조 기반
    def __init__(self, in_channels):
        super().__init__()
        self.query = nn.Conv2d(in_channels, in_channels // 8, 1)
        self.key = nn.Conv2d(in_channels, in_channels // 8, 1)
        self.value = nn.Conv2d(in_channels, in_channels, 1)
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        B, C, H, W = x.size()
        query = self.query(x).view(B, -1, H * W).permute(0, 2, 1)
        key = self.key(x).view(B, -1, H * W)
        energy = torch.bmm(query, key)
        attention = F.softmax(energy, dim=-1)

        value = self.value(x).view(B, -1, H * W)
        out = torch.bmm(value, attention.permute(0, 2, 1)).view(B, C, H, W)

        return self.gamma * out + x


In [20]:
# Generator용 ResBlock (Upsampling 포함)
class GenResBlock(nn.Module):
    def __init__(self, in_ch, out_ch, num_classes):
        super().__init__()
        # 클래스 조건을 받아 BatchNorm에 적용
        self.cbn1 = ConditionalBatchNorm(in_ch, num_classes)
        self.relu1 = nn.ReLU()
        self.upsample = nn.Upsample(scale_factor=2) # 해상도 2배 증가
 
        self.conv1 = nn.utils.spectral_norm(nn.Conv2d(in_ch, out_ch, 3, 1, 1))
        self.cbn2 = ConditionalBatchNorm(out_ch, num_classes)
        self.relu2 = nn.ReLU()
        self.conv2 = nn.utils.spectral_norm(nn.Conv2d(out_ch, out_ch, 3, 1, 1))
        
        # 채널 크기가 다르면 skip 연결을 맞춰주는 1x1 conv 추가
        self.bypass = nn.utils.spectral_norm(nn.Conv2d(in_ch, out_ch, 1)) if in_ch != out_ch else None

    def forward(self, x, y):
        out = self.relu1(self.cbn1(x, y))
        out = self.upsample(out)
        out = self.conv1(out)
        out = self.relu2(self.cbn2(out, y))
        out = self.conv2(out)

        # skip connection (업샘플 + 채널 정렬)
        skip = self.upsample(x)
        if self.bypass:
            skip = self.bypass(skip)
        return out + skip


In [21]:
# Discriminator용 ResBlock (다운샘플링 포함)
class DiscResBlock(nn.Module):
    def __init__(self, in_ch, out_ch, downsample=True):
        super().__init__()
        # Spectral Normalization이 적용된 Conv2d
        self.conv1 = nn.utils.spectral_norm(nn.Conv2d(in_ch, out_ch, 3, 1, 1))
        self.conv2 = nn.utils.spectral_norm(nn.Conv2d(out_ch, out_ch, 3, 1, 1))
        self.activation = nn.LeakyReLU(0.2)
        self.downsample = downsample # 다운샘플링 여부

        self.bypass = nn.Sequential()
        if in_ch != out_ch or downsample:
            # 잔차 연결 시 채널/해상도 맞춰주는 1x1 conv
            self.bypass = nn.utils.spectral_norm(nn.Conv2d(in_ch, out_ch, 1))

    def forward(self, x):
        # 메인 경로: conv → activation → conv
        out = self.activation(self.conv1(x))
        out = self.activation(self.conv2(out))
        if self.downsample:
            out = F.avg_pool2d(out, 2) # 2x2 average pooling
        
        # 스킵 경로
        skip = self.bypass(x)
        if self.downsample:
            skip = F.avg_pool2d(skip, 2)
        return out + skip # 잔차 연결


In [22]:
# Generator
class Generator(nn.Module):
    def __init__(self, z_dim=128, class_dim=1000, ch=64):
        super().__init__()
        # latent vector z를 4x4 feature map으로 변환 (시작점)
        self.linear = nn.utils.spectral_norm(nn.Linear(z_dim, ch * 16 * 4 * 4))\
        
        # ResBlock 순차적으로 업샘플링
        self.block1 = GenResBlock(ch * 16, ch * 16, class_dim)  # 4x4 → 8x8
        self.block2 = GenResBlock(ch * 16, ch * 8, class_dim)   # 8x8 → 16x16
        self.block3 = GenResBlock(ch * 8, ch * 4, class_dim)    # 16x16 → 32x32
        
        # Self-Attention (논문에서 32x32에 적용)
        self.attn = SelfAttention(ch * 4)                       # optional: 32x32
        self.block4 = GenResBlock(ch * 4, ch * 2, class_dim)    # 32x32 → 64x64

        self.bn = nn.BatchNorm2d(ch * 2) # 마지막 정규화
        self.relu = nn.ReLU()
        self.to_rgb = nn.Conv2d(ch * 2, 3, 3, 1, 1) # RGB 이미지로 변환

    def forward(self, z, y):
        # z를 4x4 feature map으로 변환
        out = self.linear(z).view(z.size(0), -1, 4, 4)

        # 클래스 조건 포함하여 블록 통과
        out = self.block1(out, y) # 8x8
        out = self.block2(out, y) # 16x16
        out = self.block3(out, y) # 32x32
        out = self.attn(out) # Attention
        out = self.block4(out, y) # 64x64
        out = self.relu(self.bn(out))
        return torch.tanh(self.to_rgb(out))


In [23]:
# Discriminator
class Discriminator(nn.Module):
    def __init__(self, class_dim=1000, ch=64):
        super().__init__()
        # ResBlock을 통해 다운샘플링 반복
        self.block1 = DiscResBlock(3, ch, downsample=True)       # 64x64 → 32x32
        self.block2 = DiscResBlock(ch, ch * 2, downsample=True)  # 32x32 → 16x16
        # # Attention 레이어: 16x16 또는 8x8에서 사용 가능
        self.attn = SelfAttention(ch * 2)                         # optional
        self.block3 = DiscResBlock(ch * 2, ch * 4, downsample=True) # 16x16 → 8x8
        self.block4 = DiscResBlock(ch * 4, ch * 8, downsample=True) # 8x8 → 4x4
        self.block5 = DiscResBlock(ch * 8, ch * 16, downsample=True) # 4x4 → 2x2

        self.relu = nn.LeakyReLU(0.2)
        # 판별기 출력용 선형 레이어
        self.linear = nn.utils.spectral_norm(nn.Linear(ch * 16, 1))
        # 클래스 임베딩 벡터와 내부 피처 간 내적을 통해 클래스 조건 반영
        self.embed = nn.utils.spectral_norm(nn.Embedding(class_dim, ch * 16))

    def forward(self, x, y):
        out = self.block1(x) # 64→32
        out = self.block2(out) # 32→16
        out = self.attn(out) # attention
        out = self.block3(out) # 16→8
        out = self.block4(out) # 8→4
        out = self.block5(out) # 4→2

        out = self.relu(out)
        out = out.sum(dim=[2, 3])  # Global sum pooling (2x2 → 벡터)

        # 클래스 정보와 임베딩을 내적 (projection discriminator)
        out_linear = self.linear(out)
        out_embed = (self.embed(y) * out).sum(dim=1, keepdim=True)
        return out_linear + out_embed # 최종 score 출력


### <b>6. ViT VQ-GAN<b>

In [24]:
# VQ-VAE: 이미지를 discrete latent 토큰으로 인코딩/디코딩 (vector quantization 사용)
# GAN loss: 디코더 퀄리티 향상을 위한 adversarial loss 사용
# ViT: 디코더(또는 discriminator)에 Vision Transformer를 도입
# Perceptual loss: LPIPS 등 시각적 유사도 유지 목적의 추가 loss 사용

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Encoder: 이미지를 latent feature로 압축
class Encoder(nn.Module):
    def __init__(self, in_channels=3, hidden_dim=256):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels, hidden_dim, 4, 2, 1),  # 64 → 32
            nn.ReLU(),
            nn.Conv2d(hidden_dim, hidden_dim, 4, 2, 1),   # 32 → 16
            nn.ReLU(),
            nn.Conv2d(hidden_dim, hidden_dim, 3, 1, 1)    # 16x16 latent (유지)
        )

    def forward(self, x):
        return self.model(x) # 이미지 → latent feature map

# Vector Quantizer: vector quantization (codebook lookup)
class VectorQuantizer(nn.Module):
    def __init__(self, num_codes=512, code_dim=256):
        super().__init__()
        # embedding table: [num_codes, code_dim]
        self.codebook = nn.Embedding(num_codes, code_dim)
        self.codebook.weight.data.uniform_(-1 / num_codes, 1 / num_codes)

    def forward(self, z):
        B, C, H, W = z.shape
        # [B, C, H, W] → [B*H*W, C]: spatial 위치별로 펼침
        z_flat = z.permute(0, 2, 3, 1).reshape(-1, C) 

        # 각 벡터와 코드북 간 L2 거리 계산
        distances = (
            z_flat.pow(2).sum(1, keepdim=True)
            - 2 * z_flat @ self.codebook.weight.t()
            + self.codebook.weight.pow(2).sum(1)
        )  # [BHW, num_codes]
        
        indices = torch.argmin(distances, dim=1)  # 가장 가까운 코드 index 선택
        # 선택된 코드북 벡터로 대체 → quantized output
        quantized = self.codebook(indices).view(B, H, W, C).permute(0, 3, 1, 2)

        commitment_loss = F.mse_loss(quantized.detach(), z) # Loss 1: z와 quantized의 차이 (commitment loss)
        codebook_loss = F.mse_loss(quantized, z.detach()) # Loss 2: codebook update (codebook이 z에 가까워지도록)
        quantized = z + (quantized - z).detach() # Straight-through estimator: backward는 z 경로로 통과
        # total loss, quantized output, index map 반환
        return quantized, commitment_loss + codebook_loss, indices.view(B, H, W) 


# Decoder: quantized feature → 원본 이미지 복원

class Decoder(nn.Module):
    def __init__(self, hidden_dim=256, out_channels=3):
        super().__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(hidden_dim, hidden_dim, 4, 2, 1),  # 16 → 32
            nn.ReLU(),
            nn.ConvTranspose2d(hidden_dim, hidden_dim, 4, 2, 1),  # 32 → 64
            nn.ReLU(),
            nn.Conv2d(hidden_dim, out_channels, 3, 1, 1),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)

# ViT Discriminator: Vision Transformer 기반 판별기

from torchvision.models.vision_transformer import vit_b_16

class ViTDiscriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.vit = vit_b_16(pretrained=False) # ViT-Base (pretrained X): patch size 16x16
        self.vit.heads = nn.Linear(self.vit.heads.in_features, 1) # 출력층 수정: 이진 분류 (진짜/가짜)

    def forward(self, x):
        return self.vit(x).squeeze() # [B, 1] → [B]


In [26]:
# VQGAN Wrapper: encoder + quantizer + decoder
class VQGAN(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = Encoder() # 이미지 → latent
        self.quantizer = VectorQuantizer() # vector quantization
        self.decoder = Decoder() # latent → 이미지 복원

    def forward(self, x):
        z_e = self.encoder(x) # encoder output
        z_q, q_loss, _ = self.quantizer(z_e) # vector quantization
        x_recon = self.decoder(z_q) # decoder
        return x_recon, q_loss


In [27]:
# Loss 함수: 재구성 + VQ + GAN 통합
def vqgan_loss(x, x_recon, q_loss, d_real, d_fake):
    # Reconstruction loss (L2) (perceptual loss로 대체 가능)
    recon_loss = F.mse_loss(x_recon, x)

    # Generator loss = 재구성 + VQ loss - GAN reward
    g_loss = recon_loss + 1e-6 * q_loss - d_fake.mean()

    # Discriminator: 진짜는 1, 가짜는 0 (hinge loss)
    # Discriminator hinge loss
    d_loss = F.relu(1.0 - d_real).mean() + F.relu(1.0 + d_fake).mean()

    return g_loss, d_loss
